In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

%cd '/content/gdrive/My Drive/LDS0_K287_Online_PhamMinhQuang/Project2/'

Mounted at /content/gdrive
/content/gdrive/My Drive/LDS0_K287_Online_PhamMinhQuang/Project2


In [ ]:
!pip install underthesea

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 40.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from underthesea import word_tokenize, pos_tag, sent_tokenize
import warnings
from gensim import corpora, models, similarities
import re

In [ ]:
STOP_WORD_FILE = 'vietnamese-stopwords.txt'

In [ ]:
with open(STOP_WORD_FILE, 'r', encoding='utf-8') as file:
    stop_words = file.read()
stop_words = stop_words.split('\n')

In [ ]:
df = pd.read_csv('ProductRaw.csv')[['name', 'description']]

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
df['products'] = df['name'] + ' ' + df['description']

In [ ]:
# ở source này em chọn name và description để phân tích
df = df[['products']].astype(str)

In [ ]:
df.head()

products
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     Tai nghe Bluetooth Inpods 12 - Cảm biến vân tay, chống nước,màu sắc đa dạng- 5 màu sắc lựa chọn THÔNG TIN CHI TIẾT\nDung lượng pin 300\nThời gian pin - Thời gian nghe nhạc liên tục từ 2.5-4h - Thời gian sạc đầy chỉ khoảng 60p - Thời gian chờ lên tới 140 giờ\nBluetooth 5\nThương hiệu OEM\nXuất xứ thương hiệu Trung Quốc\nĐộ nhạy cảm biến vân tay\nModel i12\nLoại Jack cắm USB Cable\nTrọng lượng 300g\nThời gian sử dụng\n- Thời gian nghe nhạc liên tục từ 2.5-4h\nSKU 4096608751631\nMÔ TẢ SẢN PHẨM\nINPOD 12 là phiên bản nâng cấp mới nhất , tai nghe Bluetooth 5.0 có thiết kế tỉ lệ chuẩn 1:1 với tai Airpod chính hãng\nLược bỏ nút bấm trên thân tai thay vào đó là nút cảm ứng sử dụng dễ dàng và thuận tiện hơn\nThao tác nhận cuộc gọi đến, chuyển nhạc...một cách dễ dàng chỉ bằng 1 chạm\nBluetooth 5.0 mới nhất cho kết nối vô cùng ổn định\n- Bluetooth 5.0 mới nhất cho kết nối vô cùng ổn định\n- 2 tai nghe tự kết nối với nhau khi được lấy ra từ dock sạc\n- Chất âm và thời lượng pin được cải thiện tối ưu\n- Dock sạc tiện lợi cho tới 4 lần sạc đầy pin tai nghe\n-Các thiết bị tương thích: các thiết bị Bluetooth như cho Xiaomi, Samsung, Apple, Huawei, OPPO, vivo, Lenovo, tablet, v. v. \n- Thời gian nghe nhạc liên tục từ 2.5-4h\n- Thời gian sạc đầy chỉ khoảng 60p\n- Thời gian chờ lên tới 140 giờ\nGiá sản phẩm trên Tiki đã bao gồm thuế theo luật hiện hành. Tuy 

### data exploration

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4404 entries, 0 to 4403
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   products  4404 non-null   object
dtypes: object(1)
memory usage: 34.5+ KB


In [ ]:
df.duplicated().sum()

33

In [ ]:
#drop dup
df.drop_duplicates(subset=['products'], inplace = True, ignore_index = True)

In [ ]:
# word_tokenize
df["products_wt"]=df["products"].apply(lambda x: word_tokenize(x, format="text"))

In [ ]:
df.head()

products  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     Tai nghe Bluetooth Inpods 12 - Cảm biến vân tay, chống nước,màu sắc đa dạng- 5 màu sắc lựa chọn THÔNG TIN CHI TIẾT\nDung lượng pin 300\nThời gian pin - Thời gian nghe nhạc liên tục từ 2.5-4h - Thời gian sạc đầy chỉ khoảng 60p - Thời gian chờ lên tới 140 giờ\nBluetooth 5\nThương hiệu OEM\nXuất xứ thương hiệu Trung Quốc\nĐộ nhạy cảm biến vân tay\nModel i12\nLoại Jack cắm USB Cable\nTrọng lượng 300g\nThời gian sử dụng\n- Thời gian nghe nhạc liên tục từ 2.5-4h\nSKU 4096608751631\nMÔ TẢ SẢN PHẨM\nINPOD 12 là phiên bản nâng cấp mới nhất , tai nghe Bluetooth 5.0 có thiết kế tỉ lệ chuẩn 1:1 với tai Airpod chính hãng\nLược bỏ nút bấm trên thân tai thay vào đó là nút cảm ứng sử dụng dễ dàng và thuận tiện hơn\nThao tác nhận cuộc gọi đến, chuyển nhạc...một cách dễ dàng chỉ bằng 1 chạm\nBluetooth 5.0 mới nhất cho kết nối vô cùng ổn định\n- Bluetooth 5.0 mới nhất cho kết nối vô cùng ổn định\n- 2 tai nghe tự kết nối với nhau khi được lấy ra từ dock sạc\n- Chất âm và thời lượng pin được cải thiện tối ưu\n- Dock sạc tiện lợi cho tới 4 lần sạc đầy pin tai nghe\n-Các thiết bị tương thích: các thiết bị Bluetooth như cho Xiaomi, Samsung, Apple, Huawei, OPPO, vivo, Lenovo, tablet, v. v. \n- Thời gian nghe nhạc liên tục từ 2.5-4h\n- Thời gian sạc đầy chỉ khoảng 60p\n- Thời gian chờ lên tới 140 giờ\nGiá sản phẩm trên Tiki đã bao gồm thuế theo luật hiện hành. T

### cosine_similarity

In [ ]:
# Tokenize(split) the sentences into words
products_gem = [[text for text in x.split()] for x in df.products_wt]

In [ ]:
products_gem[:2]

[['Tai',
  'nghe',
  'Bluetooth_Inpods',
  '12',
  '-',
  'Cảm_biến',
  'vân',
  'tay',
  ',',
  'chống',
  'nước',
  ',',
  'màu_sắc',
  'đa_dạng',
  '-',
  '5',
  'màu_sắc',
  'lựa_chọn',
  'THÔNG_TIN',
  'CHI_TIẾT',
  'Dung_lượng',
  'pin',
  '300',
  'Thời_gian',
  'pin',
  '-',
  'Thời_gian',
  'nghe',
  'nhạc',
  'liên_tục',
  'từ',
  '2.5',
  '-',
  '4',
  'h',
  '-',
  'Thời_gian',
  'sạc',
  'đầy',
  'chỉ',
  'khoảng',
  '60',
  'p',
  '-',
  'Thời_gian',
  'chờ',
  'lên',
  'tới',
  '140',
  'giờ',
  'Bluetooth',
  '5',
  'Thương_hiệu',
  'OEM',
  'Xuất_xứ',
  'thương_hiệu',
  'Trung_Quốc',
  'Độ',
  'nhạy_cảm_biến',
  'vân',
  'tay',
  'Model_i12',
  'Loại',
  'Jack_cắm',
  'USB_Cable',
  'Trọng_lượng',
  '300',
  'g',
  'Thời_gian',
  'sử_dụng',
  '-',
  'Thời_gian',
  'nghe',
  'nhạc',
  'liên_tục',
  'từ',
  '2.5',
  '-',
  '4',
  'h',
  'SKU_4096608751631',
  'MÔ_TẢ',
  'SẢN_PHẨM',
  'INPOD',
  '12',
  'là',
  'phiên_bản',
  'nâng_cấp',
  'mới',
  'nhất',
  ',',
  'tai',

In [ ]:
len(products_gem)

4371

In [ ]:
# phần này viết thành function, bỏ bớt những từ ko cần thiết
def removeUnnecessariness(products_gem):
  products_gem_re = [[re.sub('[0-9]+','', e) for e in text] for text in products_gem] # bỏ số
  products_gem_re = [[t.lower() for t in text ] for text in  products_gem_re]
  products_gem_re = [[t for t in text if not t in ['thông_tin', 'chi_tiết', '', ' ', ',', '.', '...', '-',':', ';', '?', '%', '(', ')', '+', '/', 'g', 'ml',
  'tới','tùy','vào','từng','loại','mà','như','hàng','có','không', 'dây','hơn','với','và','cho','có','rất','nhau','bạn','sẽ','theo']] for text in  products_gem_re] # ký tự đặc biệt
  products_gem_re = [[t for t in text if not t in stop_words] for text in products_gem_re] # stopword
  return products_gem_re
products_gem_re = removeUnnecessariness(products_gem)

In [ ]:
df['products_wt'] = products_gem_re

In [ ]:
df[['products_wt']].head()

,products_wt
0,"[tai, bluetooth_inpods, cảm_biến, vân, chống, màu_sắc, đa_dạng, màu_sắc, lựa_chọn, dung_lượng, pin, pin, nhạc, liên_tục, h, sạc, p, chờ, bluetooth, thương_hiệu, oem, xuất_xứ, thương_hiệu, trung_quốc, độ, nhạy_cảm_biến, vân, model_i, jack_cắm, usb_cable, trọng_lượng, nhạc, liên_tục, h, sku_, mô_tả, sản_phẩm, inpod, phiên_bản, nâng_cấp, tai, bluetooth_., thiết_kế, tỉ_lệ, chuẩn, tai_airpod, hãng, lược, nút, bấm, thân_tai, thay, nút, cảm_ứng, dễ_dàng, thuận_tiện, thao_tác, cuộc_gọi, chuyển_nhạc, dễ_dàng, chạm, bluetooth_., kết_nối, vô_cùng, ổn_định, bluetooth_., kết_nối, vô_cùng, ổn_định, tai, kết_nối, dock, sạc, chất_âm, thời_lượng, pin, cải_thiện, tối_ưu, dock, sạc, tiện_lợi, sạc, pin_tai, thiết_bị, tương_thích, thiết_bị, bluetooth, xiaomi, samsung, apple, huawei, oppo, vivo, lenovo, tablet, v, v, nhạc, liên_tục, h, ...]"
1,"[tai, bluetooth, f, true_wireless, dock_sạc, led_báo, pin_kép, dung_lượng, pin, mah, pin, nhạc, đàm_thoại, bluetooth_v, thương_hiệu, oem, xuất_xứ, thương_hiệu, trung_quốc, độ_nhạy, cao_model, f_loại, jack_cắm, bluetooth, trọng_lượng, tần_số, hz, nhạc, đàm_thoại, đàm_thoại, sku_, mô_tả, sản_phẩm, chip, bluetooth_., mạnh_mẽ, kết_nối, ổn_định, kết_nối, m, tiết_kiệm, năng_lượng, pin, khỏi_động, nhấc, hộp_sạc, tự_động, kết_nối, bluetooth, điện_thoại, trạng_thai, sẵn_sàng, ngắt_kế, nối, hộp_sạc, hoạt_động, phạm_vi, m, công_nghệ_khử, tiếng, ồn, âm_thanh, trung_thực, chất_lượng, cvc_., dung_lượng, pin, tai, mah, trọng_lượng, tai, nhẹ, khít, cảm_giác, thoải_mái, dung, lương, dock_sạc, mah, sạc, tai, sạc, tai, mà_còn, sạc, điện_thoại, nhạc, đàm_thoại, cáp_sạc, microusb, màn_hình, đèn_hiển_thị, chế_độ, sạc, báo, pin_yếu, dễ_dàng, kết_nối, thiết_bị, kết_nối, ...]"
2,"[chuột, dây_logitech, m_silent, hàng_chính, hãng, thương_hiệu, logitech_độ, phân_giải, quang_học, dpi, model_m, lưu_ý_cách, kéo_dài, tuổi_thọ, pin, chuột, on, off, làm_việc, usb, tín_hiệu, receiver, chuột, bàn, chuột, màu, tối, đục, mờ, mắt, cảm_ứng, tiêu_hao, năng_lượng, chuột, tốn, năng_lượng, bề_mặt, tối, đục, gồ, ghề, bàn, chuột, màu, mượt, tối_ưu, năng_lượng, pin, dung_lượng, hãng, hiệu_suất, pin, xuất_xứ, trung_quốc, sku_, mô_tả, sản_phẩm_độ, phân_giải, dpi, nút, nhấn, cơ_bản, gạt, on, of, kết_nối, ._ghz, kết_nối, m, pin_aa, cảm_biến_quang, giúp, kéo_dài, tuổi_thọ, pin, thiết_kế, sang_trọng_chuột, dây_logitech, m_silent, thiết_kế, đường_cong, uốn_lượn, bo_tròn, ôm, sát_lòng, bàn_tay, giúp, bớt, đau_mỏi, cổ_tay, ngón, hai, thân, chuột, khối_hình, tam_giác, sắp_xếp, liền, kề, độ, ...]"
3,"[loa, bluetooth, ._kiêm, đồng_hồ, báo_thức, [_[, ]_], robot, hàng_chính, hãng, thương_hiệu, acome, xuất_xứ, thương_hiệu, indonesia, xuất_xứ, trung_quốc, sku_, mô_tả, sản_phẩm, xuất_xứ, thương_hiệu, indonesia, hãng, phân_phối, độc_quyền, thương_hiệu, acome, thiết_kế, hồng_kông, pin, mah, playtime, liên_tục, h, loa, bluetooth_., thông_số, •_cố, đồng_hồ, báo_thức, kính_tráng, gương, màn, led_•, làm_việc, m_•, công_suất, đầu_ra, w_•, dải_tần, đáp_ứng, h_z, kh, z_•, tỷ_lệ, tín_hiệu, nhiễu, ≥, db_•, dung_lượng, pin, mah, v_•, •, sạc, •_trọng_lượng, g_ram, tính_năng_loa, bluetooth_., kết_nối, nhanh_công_nghệ, bluetooth_., tốc_độ, kết_nối, ngắt_quãng, thiết_kế, kiêm, đồng_hồ, báo_thức, màn_hình_tráng, gương, hiển_thị_led, dung_lượng, pin, mah, công_nghệ, sạc, thông_minh, loa, sạc, nhạc, trình, phát_nhạc, đáp_ứng, nhu_cầu, đa_dạng, phát, buetooth, phát, ...]"
4,"[tai, bluetooth_apple, airpods_pro, true_wireless, mwp, hàng_chính, hãng, vn, a_thông_tin, thương_hiệu, apple, xuất_xứ, thương_hiệu, mỹ_kích_thước, tai, mm, hộp_sạc, mm, model_mwp, trọng_lượng, tai, ._g, hộp_sạc, sku_, mô_tả, sản_phẩm, thiết_kế, thời_trang, gọn, khả_năng, chống, ồn, –, âm_thanh, vượt_bậc, tái_tạo, âm_thanh, sống_động, kết_nối, bluetooth_tiện, lợi_trang, tính_năng, theo_dõi, sức_khỏe, thời_lượng, pin, cải_thiện, tiếng, tương_thích, tối_đa, thiết_bị, apple_airpods, pro_sở_hữu, thiết_kế, gọn, tai_nghe, bluetooth_apple, airpods_pro, true_wireless, mwp, thiết_kế, tai, in-ear, thay_vì, earbud,

In [ ]:
df['products_wt'] = df['products_wt'].apply(lambda x: ' '.join(map(str,x)))

In [ ]:
tf = TfidfVectorizer(analyzer='word', min_df=0, stop_words=stop_words)

In [ ]:
tfidf_matrix = tf.fit_transform(df.products_wt)

In [ ]:
cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
cosine_similarities

array([[1.        , 0.33179828, 0.07149014, ..., 0.01660965, 0.01355677,
        0.04419543],
       [0.33179828, 1.        , 0.05211178, ..., 0.01859081, 0.02496951,
        0.05628655],
       [0.07149014, 0.05211178, 1.        , ..., 0.0290136 , 0.02080834,
        0.03231751],
       ...,
       [0.01660965, 0.01859081, 0.0290136 , ..., 1.        , 0.0061089 ,
        0.03393839],
       [0.01355677, 0.02496951, 0.02080834, ..., 0.0061089 , 1.        ,
        0.05413531],
       [0.04419543, 0.05628655, 0.03231751, ..., 0.03393839, 0.05413531,
        1.        ]])

In [ ]:
df_show = pd.DataFrame(cosine_similarities)
df_show.head(9)

,0,1,2,3,4,5,6,7,8,9,...,4361,4362,4363,4364,4365,4366,4367,4368,4369,4370
0,1.000000,0.331798,0.071490,0.170450,0.157136,0.138049,0.185765,0.134613,0.141008,0.046529,...,0.115979,0.027243,0.023122,0.020604,0.037810,0.013212,0.038304,0.016610,0.013557,0.044195
1,0.331798,1.000000,0.052112,0.121762,0.351712,0.212953,0.326346,0.087040,0.307999,0.071003,...,0.139482,0.014767,0.036774,0.013276,0.050167,0.030937,0.052710,0.018591,0.024970,0.056287
2,0.071490,0.052112,1.000000,0.053660,0.051421,0.026025,0.040233,0.045886,0.022034,0.258121,...,0.062762,0.032325,0.014028,0.032042,0.014048,0.038979,0.043128,0.029014,0.020808,0.032318
3,0.170450,0.121762,0.053660,1.000000,0.038462,0.031977,0.029187,0.107303,0.026452,0.036105,...,0.134045,0.014319,0.030112,0.027430,0.082591,0.019211,0.041884,0.024526,0.042596,0.053854
4,0.157136,0.351712,0.051421,0.038462,1.000000,0.238448,0.264314,0.034224,0.250801,0.052717,...,0.071982,0.027242,0.033078,0.017773,0.056819,0.051693,0.092514,0.021056,0.006163,0.099215
5,0.138049,0.212953,0.026025,0.031977,0.238448,1.000000,0.157229,0.047641,0.122195,0.041246,...,0.036365,0.017216,0.008797,0.013305,0.022145,0.027451,0.046525,0.013977,0.010103,0.038655
6,0.185765,0.326346,0.040233,0.029187,0.264314,0.157229,1.000000,0.033184,0.290075,0.043974,...,0.079962,0.029140,0.105075,0.029213,0.049138,0.049788,0.099914,0.018976,0.034498,0.164646
7,0.134613,0.087040,0.045886,0.107303,0.034224,0.047641,0.033184,1.000000,0.016258,0.032227,...,0.083751,0.012920,0.015251,0.013765,0.012960,0.020571,0.040722,0.016749,0.022048,0.068626
8,0.141008,0.307999,0.022034,0.026452,0.250801,0.122195,0.290075,0.016258,1.000000,0.025447,...,0.064572,0.020584,0.060604,0.015305,0.054330,0.022229,0.065476,0.016270,0.011504,0.108503


### test 1 id bất kỳ lấy ra n similarities

In [ ]:
def findfindSimilarityExisting(cosine_similarities, df, n, input_idx):
  results = []
  for idx, row in df.iterrows():
      similar_indices = (-cosine_similarities[idx]).argsort()[1: n+1]
      similar_items = [(cosine_similarities[idx][i]) for i in similar_indices]
      similar_items = [(cosine_similarities[idx][i], df.index[i]) for i in similar_indices]
      if idx == input_idx:
        results = similar_items
  for i in range(0, len(results)):
    print('match {}: '.format(i+1), df.values[results[i][1]])
findfindSimilarityExisting(cosine_similarities, df, 4, 24)

match 1:  ['Tai nghe bluetooth không dây i12 TWS 5.0, thiết kế nhỏ gọn, chất lượng âm thanh nổi HIFI, chức năng giảm tạp âm THÔNG TIN CHI TIẾT\nThương hiệu OEM\nXuất xứ thương hiệu Trung Quốc\nXuất xứ Trung Quốc\nSKU 9129406802729\nMÔ TẢ SẢN PHẨM\nDung lượng pin cục sạc: 400AH\nDung lượng pin tai nghe:40mAh\nThời gian sạc: 70 phút\nĐiện áp hộp sạc: 5V\nThời gian sạc hộp sạc: 1 giờ\nHộp sạc sạc tai nghe được 3 lần\nKhi sạc tai nghe，đèn sáng màu đỏ, khi sạc đầy đèn tự động ngắt.\nThiết kế thời thượng\nKết nối Bluetooth 5.0 vô cùng ổn định, 2 tai tự kết nối với nhau khi lấy ra từ dock sạch\nDock sạc tiện lợi cho tới 3 lần sạc đầy pin tai nghe, \nThời lượng pin tới hơn 2,5 h nghe nhạc liên tục mà chỉ mất 1h để sạc đầy cho tai nghe\nChất âm trầm ấm, phù hợp nhiều thể loại nhạc, chơi game,\nĐiều chỉnh âm lượng to nhỏ và next bài trực tiếp trên tai nghe.\nĐàm thoại cùng lúc cả 2 tai.\nBước 1: Lấy 2 tai nghe ra khỏi dock sạc\nBước 2: Hai tai nghe sẽ tự nhấp nháy đèn để tự liên kết với nhau\nBư

### test tên mặt hàng bất kỳ

In [ ]:
df = pd.read_csv('ProductRaw.csv')[['name', 'description']]
df['products'] = df['name'] + ' ' + df['description']
df = df[['products']].astype(str)
df.drop_duplicates(subset=['products'], inplace = True, ignore_index = True)

In [ ]:
def findSimilarityArbitrary(input_text, df):
  df_newline = pd.DataFrame({"products":[input_text]})
  df = df.append(df_newline, ignore_index = True)
  df["products_wt"]=df["products"].apply(lambda x: word_tokenize(x, format="text"))
  products_gem = [[text for text in x.split()] for x in df.products_wt]
  products_gem_re = removeUnnecessariness(products_gem)
  df['products_wt'] = products_gem_re
  df['products_wt'] = df['products_wt'].apply(lambda x: ' '.join(map(str,x)))
  tf = TfidfVectorizer(analyzer='word', min_df=0, stop_words=stop_words)
  tfidf_matrix = tf.fit_transform(df.products_wt)
  cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)
  findfindSimilarityExisting(cosine_similarities, df, 4, len(df)-1)

In [ ]:
findSimilarityArbitrary('Điện thoại Samsung Galaxy A53 5G (8GB/128GB) - Hàng chính hãng', df)

<ipython-input-40-3ffa360a026a>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_newline, ignore_index = True)


match 1:  ['Điện Thoại Samsung Galaxy S21 Ultra 5G (12GB/128GB) - Hàng Chính Hãng THÔNG TIN CHI TIẾT\nDung lượng pin 5000 mAh\nBluetooth Bluetooth 5.1\nThương hiệu Samsung\nXuất xứ thương hiệu Hàn Quốc\nChip set Exynos 2100\nNFC Có\nFM radio Không\nGhi âm Có\nSố sim 2\nLoại pin 5.000 mAh\nNghe nhạc Có\nXuất xứ Hàn Quốc\nCổng sạc USB Type-C 3.2\nRAM 16 GB\nROM 512 GB\nKích thước màn hình 6.8 inch\nXem phim Có\nSKU 8549842564143\nMÔ TẢ SẢN PHẨM\nThiết kế ấn tượng\nTheo như những hình ảnh rò rỉ lan truyền qua nhiều nguồn, thì dòng sản phẩm mới sắp ra mắt Samsung Galaxy S21 Ultra sẽ sở hữu khung vỏ kim loại và hai mặt kính trước sau nịnh mắt – ngôn ngữ thiết kế chủ đạo trên các dòng smartphone cao cấp nhất hiện tại.\nMàn hình Full HD+, tần số quét 120Hz mượt mà\nTheo các nguồn tin cho hay thì Samsung Galaxy S21 Ultra được trang bị màn hình full HD+ siêu sắc nét với tần số quét 120Hz mượt mà.\nHiệu năng mạnh mẽ\nSamsung Galaxy S21 Ultra với chip Exynos 2100 hoặc Snapdragon 875 cho tốc độ

Nhận xét: Kết quả recommend khá đúng nhãn hiệu điện thoại và dòng điện thoại, kết quả số 4 ko chính xác lắm